<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson OpenScale - Headless Subscription

# Pipeline variables

In [ ]:
# Deploymentspace ID
space_uid = 'c4238e9c-1cbd-4776-aa6e-4f6b1f865ed1'

# DeploymentID of the model that will be monitored
# TODO: make 04_deploy_models emit this and then use that URL
deployment_uid = '66271495-2e3e-4ab2-ae2f-521330555bdf'

# Credentials

In [ ]:
from ibm_watson_machine_learning import APIClient
import os
from dotenv import load_dotenv
# Loading Variables and Utils from common python file
import vars_and_utils as vars_and_utils

load_dotenv()

token = os.environ['USER_ACCESS_TOKEN']
cpd_technical_user = os.environ['cpd_technical_user']
cpd_technical_user_password = os.environ['cpd_technical_user_password']
cpd_url = os.environ['cpd_url']



WML_CREDENTIALS = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.7"
}



WOS_CREDENTIALS = {
    "url": cpd_url,
    "username": cpd_technical_user,
    "password": cpd_technical_user_password,
    "version": "4.7"
}

In [ ]:
print(WOS_CREDENTIALS)

# WOS name definitions

In [ ]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = "Added by automated WOS configuration notebook."
SUBSCRIPTION_NAME = "AOT Initiative - Headless Subscription"

# Setup - Package installation <a name="setup"></a>

In [ ]:
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade "ibm-metrics-plugin>=4.6.4.0"

# Imports

In [ ]:
import pandas as pd
import tarfile
from io import BytesIO

from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator, CloudPakForDataAuthenticator

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Get training data statistics

### Get the training data

In [ ]:
data_df = vars_and_utils.load_data(vars_and_utils.raw_data_path)
data_df.head()

In [ ]:
# import itc_utils.flight_service as itcfs

# readClient = itcfs.get_flight_client()

# nb_data_request = {
#     'data_name': """german_credit_data_biased_training.csv""",
#     'interaction_properties': {
#         #'row_limit': 500,
#         'infer_schema': 'true',
#         'infer_as_varchar': 'false'
#     }
# }

# flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

# data_df = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)

### Generate the training data stats

In [ ]:
from ibm_watson_openscale.utils.training_stats import TrainingStats

In [ ]:
feature_columns = data_df.drop("Risk", axis=1).columns.tolist()
cat_features = [x for x in feature_columns if data_df[x].dtype == 'object']
class_label = "Risk"
prediction_column = "prediction"
probability_column = "probability"

In [ ]:
service_configuration_support = {
    "enable_fairness": True,
    "enable_explainability": True,
    "enable_drift": True
}

fairness_attributes = [{
   "feature": "Sex", 
   "majority": [
       "male"
   ],
   "minority": [
       "female"
   ],
   "threshold": 0.8
}]

model_type = "binary"
parameters = {
    "favourable_class" :  [ "No Risk" ],
    "unfavourable_class": [ "Risk" ]
}
min_records = 100

# Generate Training stats
enable_explainability = service_configuration_support.get('enable_explainability')
enable_fairness = service_configuration_support.get('enable_fairness')
training_data_stats = None
if enable_explainability or enable_fairness:
    fairness_inputs = None
    if enable_fairness:
        fairness_inputs = {
                "fairness_attributes": fairness_attributes,
                "min_records" : min_records,
                "favourable_class" :  parameters["favourable_class"],
                "unfavourable_class": parameters["unfavourable_class"]
            }
        
input_parameters = {
    "label_column": class_label,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features,
    "fairness_inputs": fairness_inputs,
    "problem_type" : "binary",
    "prediction": prediction_column,
    "probability": "probability"
}

In [ ]:
training_stats = TrainingStats(data_df,input_parameters, explain=True, fairness=enable_explainability, drop_na=True)

In [ ]:
training_data_stats = training_stats.get_training_statistics()

In [ ]:
training_data_stats["notebook_version"] = 5.0
print(training_data_stats)

### This JSON contains the training statistics

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

## Get a instance of the OpenScale SDK client and connect to WOS datamart

Watson OpenScale uses a database to store payload and feedback logs and calculated metrics. Here we are using already configured data mart in IBM Cloud.

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

try:
    wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"])
    print("Authentication Successful")
    data_marts = wos_client.data_marts.list().result.data_marts
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))
except:
        print("ERROR: Authorization request has been rejected with message: AIQCS0002E : Not authorized to access datamart id `00000000-0000-0000-0000-000000000000`.")
        if DATAMART_ID=="00000000-0000-0000-0000-000000000000":
            DATAMART_ID=input("Please enter your datamart id to authenticate")
        print("\nTrying to authenticate with the DATAMART_ID provided..")
        wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=DATAMART_ID)
        print("Authentication Successful.")

## Remove existing service provider

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

Note: Here the service provider is created with empty credentials, meaning no endpoint. Just to demonstrate the use case were we don't need an actual end point serving requests.

In [ ]:
MLCredentials = {}
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=MLCredentials,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

## Subscriptions

This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

## Remove the existing subscription

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == '[asset] ' + SUBSCRIPTION_NAME:
        sub_model_id = subscription.metadata.id
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [ ]:
print("Data Mart ID: " + data_mart_id)
print("Service Provide ID: " + service_provider_id)
import uuid
asset_id = str(uuid.uuid4())
asset_name = '[asset] ' + SUBSCRIPTION_NAME
url = None

asset_deployment_id = str(uuid.uuid4())
asset_deployment_name = asset_name

In [ ]:
prediction_column = prediction_column
probability_columns = ['probability']
predicted_target_column = prediction_column

subscription_details = wos_client.subscriptions.add(data_mart_id,
    service_provider_id,
    asset=Asset(
        asset_id=asset_id,
        name=asset_name,
        url=url,
        asset_type=AssetTypes.MODEL,
        input_data_type=InputDataType.STRUCTURED,
        problem_type=ProblemType.BINARY_CLASSIFICATION
    ),
    deployment=None,    
    training_data_stats=training_data_stats, 
    prediction_field = prediction_column,
    predicted_target_field = predicted_target_column,
    probability_fields = probability_columns,background_mode = False
    ,deployment_name = asset_name
    ).result

subscription_id = subscription_details.metadata.id
print("Subscription id {}".format(subscription_id))

### The following code fetches the data set id, against which we would be performing the payload logging

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

## Push a payload record to setup the required schemas in the subscription

This is the location where one needs to fetch the output of the batch scoring model and construct the payload as per the OpenScale Payload Logging format.

Note : No scoring is done against the model. The PayloadRecord is constructed with the request and response from the model/deployment.

## Scoring Request Payload

In [ ]:
scoring_request =   {
        "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker",
            "Risk"
        ],
        "values": [
            [
                "no_checking",
                28,
                "outstanding_credit",
                "appliances",
                5990,
                "500_to_1000",
                "greater_7",
                5,
                "male",
                "co-applicant",
                3,
                "car_other",
                55,
                "none",
                "free",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "greater_200",
                22,
                "all_credits_paid_back",
                "car_used",
                3376,
                "less_100",
                "less_1",
                3,
                "female",
                "none",
                2,
                "car_other",
                32,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "no_checking",
                39,
                "credits_paid_to_date",
                "vacation",
                6434,
                "unknown",
                "greater_7",
                5,
                "male",
                "none",
                4,
                "car_other",
                39,
                "none",
                "own",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "0_to_200",
                20,
                "credits_paid_to_date",
                "furniture",
                2442,
                "less_100",
                "unemployed",
                3,
                "female",
                "none",
                1,
                "real_estate",
                42,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                4,
                "all_credits_paid_back",
                "education",
                4206,
                "less_100",
                "unemployed",
                1,
                "female",
                "none",
                3,
                "savings_insurance",
                27,
                "none",
                "own",
                1,
                "management_self-employed",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                23,
                "credits_paid_to_date",
                "car_used",
                2963,
                "greater_1000",
                "greater_7",
                4,
                "male",
                "none",
                4,
                "car_other",
                46,
                "none",
                "own",
                2,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                31,
                "prior_payments_delayed",
                "vacation",
                2673,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                2,
                "real_estate",
                35,
                "stores",
                "rent",
                1,
                "skilled",
                2,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                37,
                "prior_payments_delayed",
                "other",
                6971,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                54,
                "none",
                "own",
                2,
                "skilled",
                1,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                14,
                "all_credits_paid_back",
                "car_new",
                1525,
                "500_to_1000",
                "4_to_7",
                3,
                "male",
                "none",
                4,
                "real_estate",
                33,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "less_0",
                10,
                "prior_payments_delayed",
                "furniture",
                4037,
                "less_100",
                "4_to_7",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                31,
                "none",
                "rent",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ]
        ]
    }

## Scoring Response Payload

In [ ]:
scoring_response = {
    "predictions": [
        {
            "fields": [
                "prediction",
                "probability"
            ],
            "values": [
                [
                    "Risk",
                    [
                        0.104642951112211,
                        0.895357048887789
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.892112895920181,
                        0.10788710407981907
                    ]
                ],
                [
                    "Risk",
                    [
                        0.4863177905287259,
                        0.5136822094712741
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.980811537315731,
                        0.01918846268426898
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9053052561083984,
                        0.09469474389160164
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.5315146773053994,
                        0.4684853226946007
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.7689466209701616,
                        0.23105337902983833
                    ]
                ],
                [
                    "Risk",
                    [
                        0.41317664143643873,
                        0.5868233585635613
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9190247585206522,
                        0.08097524147934775
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.781841942776921,
                        0.21815805722307902
                    ]
                ]
            ]
        }
    ]
}

### Construct the payload using the scoring_request and scoring_response and then log the records

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

records_list=[]
for x in range(10):
    pl_record = PayloadRecord(request=scoring_request, response=scoring_response)
    records_list.append(pl_record)

wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)

### Make sure the records reached the payload logging table inside the OpenScale DataMart.

In [ ]:
import time
time.sleep(30)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

# Explainability Monitor Configuration
From the notebook, perform offline scoring against the customer model, create an explain archive and save this archive to data mart.

* Only Local explanations and Lime global explanations are supported.
* For contrastive explanations, as scoring is needed and because it is headless subscription without any deployment URL, contrastive explanations are not supported.

## Score the perturbations

Here, this notebook uses a credit risk model deployment in WML. This can be replaced with the scoring engine of your choice, but making sure the scoring response is in the format that OpenScale understands for monitor processing.

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.set.default_space(space_uid) # connect to deployment space

In [ ]:
perturbs_df = data_df.copy()
cols_to_remove = ["Risk"]

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    for col in cols_to_remove:
        if col in perturbs_df.columns:
            del perturbs_df[col] 

    fields = perturbs_df.columns.tolist()
    training_data_rows = perturbs_df[fields].values.tolist()

    payload_scoring = {"input_data": [{
        "fields": fields, 
        "values": [x for x in training_data_rows]
    }]}  
    return payload_scoring

def sample_scoring(no_of_records_to_score = 1):
    job_payload_ref = get_scoring_payload(no_of_records_to_score)
    score = wml_client.deployments.score(deployment_uid, meta_props=job_payload_ref)
    return job_payload_ref, scoring_response

payload_scoring, scoring_response = sample_scoring(no_of_records_to_score = 5000)

In [ ]:
fields = scoring_response['predictions'][0]['fields']
values = scoring_response['predictions'][0]['values']
scored_data = pd.DataFrame(values, columns = fields)

In [ ]:
probabilities = [pro for pro in scored_data['probability']]
predictions = [pre for pre in scored_data['prediction']]

explain_perturb_payload = {'probabilities' : probabilities,
                            'predictions' : predictions}

In [ ]:
with open('explain_scoring_response.json', 'w') as outfile:
    json.dump(explain_perturb_payload, outfile)
    
file_name = 'explain_scoring_response.tar.gz'

with tarfile.open(file_name, 'w:gz') as archive:
    archive.add('explain_scoring_response.json')

with open(file_name, 'rb') as fh:
    buf = BytesIO(fh.read())
buf = open(file_name, mode="rb").read()    

In [ ]:
with open("explain_scoring_response.tar.gz", mode="rb") as perturbations_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=perturbations_tar)

print("Uploaded perturbations scoring response archive successfully.")

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "enabled": True
}

print("Creating monitor instances...")
response = wos_client.monitor_instances.create(monitor_definition_id = None, 
                        target = None, data_mart_id = data_mart_id, training_data_stats=training_data_stats, 
                        subscription_id=subscription_id,background_mode=False, parameters = parameters)
print(response)

# Quality monitoring and feedback logging

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 100
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

## Get feedback logging dataset ID

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [ ]:
feedback_dataset_id

In [ ]:
feedback_payload = {
    "fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",
        "EmploymentDuration",
        "InstallmentPercent",
        "Sex",
        "OthersOnLoan",
        "CurrentResidenceDuration",
        "OwnsProperty",
        "Age",
        "InstallmentPlans",
        "Housing",
        "ExistingCreditsCount",
        "Job",
        "Dependents",
        "Telephone",
        "ForeignWorker",
        "Risk",
        "_original_probability",
        "_original_prediction",
        "_debiased_probability",
        "_debiased_prediction"        
    ],
    "values": [
        [
            "less_0",
            18,
            "credits_paid_to_date",
            "car_new",
            462,
            "less_100",
            "1_to_4",
            2,
            "female",
            "none",
            2,
            "savings_insurance",
            37,
            "stores",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            15,
            "prior_payments_delayed",
            "furniture",
            250,
            "less_100",
            "1_to_4",
            2,
            "male",
            "none",
            3,
            "real_estate",
            28,
            "none",
            "own",
            2,
            "skilled",
            1,
            "yes",
            "no",
            "No Risk",
            [
                0.7419002139563244,
                0.25809978604367556
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "0_to_200",
            28,
            "credits_paid_to_date",
            "retraining",
            3693,
            "less_100",
            "greater_7",
            3,
            "male",
            "none",
            2,
            "savings_insurance",
            32,
            "none",
            "own",
            1,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6935080115729353,
                0.3064919884270647
            ],
            "Risk",
            [
                0.8,
                0.2
            ],
            "Risk"
        ],
        [
            "no_checking",
            28,
            "prior_payments_delayed",
            "education",
            6235,
            "500_to_1000",
            "greater_7",
            3,
            "male",
            "none",
            3,
            "unknown",
            57,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "Risk",
            [
                0.331110352092386,
                0.668889647907614
            ],
            "Risk",
            [
                0.9,
                0.1
            ],
            "Risk"
        ],
        [
            "no_checking",
            32,
            "outstanding_credit",
            "vacation",
            9604,
            "500_to_1000",
            "greater_7",
            6,
            "male",
            "co-applicant",
            5,
            "unknown",
            57,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.1,
                0.9
            ],
            "Risk"
        ],
        [
            "no_checking",
            9,
            "prior_payments_delayed",
            "car_new",
            1032,
            "100_to_500",
            "4_to_7",
            3,
            "male",
            "none",
            4,
            "savings_insurance",
            41,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6704819620865308,
                0.32951803791346923
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            16,
            "credits_paid_to_date",
            "vacation",
            3109,
            "less_100",
            "4_to_7",
            3,
            "female",
            "none",
            1,
            "car_other",
            36,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6735810290914039,
                0.3264189709085961
            ],
            "Risk",
            [
                0.6,
                0.4
            ],
            "Risk"
        ],
        [
            "0_to_200",
            11,
            "credits_paid_to_date",
            "car_new",
            4553,
            "less_100",
            "less_1",
            3,
            "female",
            "none",
            3,
            "savings_insurance",
            22,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.637964656269084,
                0.362035343730916
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "no_checking",
            35,
            "outstanding_credit",
            "appliances",
            7138,
            "500_to_1000",
            "greater_7",
            5,
            "male",
            "co-applicant",
            4,
            "unknown",
            49,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            5,
            "all_credits_paid_back",
            "car_new",
            1523,
            "less_100",
            "unemployed",
            2,
            "female",
            "none",
            2,
            "real_estate",
            19,
            "none",
            "rent",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.7304597628653227,
                0.26954023713467745
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ]
    ]
}

In [ ]:
import urllib3, requests, json
from requests.auth import HTTPBasicAuth
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"])
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token
icp_access_token = generate_access_token()

In [ ]:
header = {
    'Content-Type': 'application/json', 
    'Authorization': 'Bearer ' + icp_access_token
}

### Store the feedback payload using the data sets API

There are two ways OpenScale APIs can be used - a) using OpenScale Python SDK b) using OpenScale REST APIs.

For any reason if in the customer environment one cannot use the SDK, then the alternative is to use the REST APIs. The below cell demostrates to invoke one such OpenScale REST API, to log the feedback records to the OpenScale DataMart.

In [ ]:
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records".format(data_mart_id, feedback_dataset_id)

In [ ]:
for x in range(10):
    response = requests.post(DATASETS_STORE_RECORDS_URL, json=feedback_payload, headers=header, verify=False)
    json_data = response.json()
    print(json_data)

### Wait for sometime, and make sure the records have reached to data sets related table.

In [ ]:
time.sleep(30)
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&include_total_count={3}".format(data_mart_id, feedback_dataset_id, 1, "true")
response = requests.get(DATASETS_STORE_RECORDS_URL, headers=header, verify=False)
json_data = response.json()
print(json_data['total_count'])

## Run Quality Monitor

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

# Drift Configuration

# Scoring function for drift configuration

In [ ]:
def score(training_data_frame):
      
    #The data type of the label column and prediction column should be same .
    #User needs to make sure that label column and prediction column array should have the same unique class labels
    
    feature_columns = list(training_data_frame.columns)
    if class_label in feature_columns:
        feature_columns.remove(class_label)
    training_data_rows = training_data_frame[feature_columns].values.tolist()
    
    payload_scoring = {
      wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
           "fields": feature_columns,
           "values": [x for x in training_data_rows]
      }]
    }

    score = wml_client.deployments.score(deployment_uid, payload_scoring)
    score_predictions = score.get('predictions')[0]

    prob_col_index = list(score_predictions.get('fields')).index(probability_column)
    predict_col_index = list(score_predictions.get('fields')).index(prediction_column)

    if prob_col_index < 0 or predict_col_index < 0:
        raise Exception("Missing prediction/probability column in the scoring response")

    import numpy as np
    probability_array = np.array([value[prob_col_index] for value in score_predictions.get('values')])
    prediction_vector = np.array([value[predict_col_index] for value in score_predictions.get('values')])

    return probability_array, prediction_vector

## Create the drift detection model archive

In [ ]:
probability_array, prediction_vector = score(data_df)

# Payload and Feedback dataset id for pushing model information into the datamart

In [ ]:
print("Payload data set id:", payload_data_set_id)
print("Feedback data set id:", feedback_dataset_id)

Authors: Moritz Scheele (moritz.scheele@ibm.com) and Ravi Chamarthy (ravi.chamarthy@in.ibm.com)